In [1]:
from pyod.models.ecod import ECOD  # ECOD detector
from pyod.models.knn import KNN   # kNN detector
from joblib import dump, load
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from loader import Loader
import os

In [2]:
clf_name = 'KNN'
# clf = ECOD()
clf = KNN()

In [3]:
loader = Loader()
if not os.path.exists('data/data.pkl'):
    loader.extract()
    df = loader.create_df_from_xlsx()
    df = loader.process_df(df)
    loader.save_df(df)
else:
    df = loader.load_df()

100%|██████████| 12872/12872 [00:04<00:00, 2708.16it/s]


In [4]:
print(df.shape)
print(df.columns)
print(df.head())

(12872, 23)
Index(['کد محور', 'نام محور', 'زمان شروع', 'زمان پایان',
       'مدت زمان کارکرد (دقیقه)', 'تعداد کل وسیله نقلیه',
       'تعداد وسیله نقلیه کلاس 1', 'تعداد وسیله نقلیه کلاس 2',
       'تعداد وسیله نقلیه کلاس 3', 'تعداد وسیله نقلیه کلاس 4',
       'تعداد وسیله نقلیه کلاس 5', 'سرعت متوسط', 'تعداد تخلف سرعت غیر مجاز',
       'تعداد تخلف فاصله غیر مجاز', 'تعداد تخلف سبقت غیر مجاز',
       'تعداد برآورد شده', 'مدت زمان کارکرد(دقیقه)', 'تعداد وسیله نقلیه سبک',
       'تعداد وسیله نقلیه سنگین', 'تعداد تخلف سرعت غیرمجاز',
       'تعداد تخلف فاصله غیرمجاز', 'تعداد تخلف سبقت غیرمجاز', 'holiday'],
      dtype='object')
   کد محور                      نام محور            زمان شروع  \
0   113201  آزادراه قم - گرمسار (چرمشهر)  1400/01/01 00:00:00   
1   113201  آزادراه قم - گرمسار (چرمشهر)  1400/01/02 00:00:00   
2   113201  آزادراه قم - گرمسار (چرمشهر)  1400/01/03 00:00:00   
3   113201  آزادراه قم - گرمسار (چرمشهر)  1400/01/04 00:00:00   
4   113201  آزادراه قم - گرمسار (چرمشهر)  1400

In [5]:
df[df['holiday'] == True]

,کد محور,نام محور,زمان شروع,زمان پایان,مدت زمان کارکرد (دقیقه),تعداد کل وسیله نقلیه,تعداد وسیله نقلیه کلاس 1,تعداد وسیله نقلیه کلاس 2,تعداد وسیله نقلیه کلاس 3,تعداد وسیله نقلیه کلاس 4,...,تعداد تخلف فاصله غیر مجاز,تعداد تخلف سبقت غیر مجاز,تعداد برآورد شده,مدت زمان کارکرد(دقیقه),تعداد وسیله نقلیه سبک,تعداد وسیله نقلیه سنگین,تعداد تخلف سرعت غیرمجاز,تعداد تخلف فاصله غیرمجاز,تعداد تخلف سبقت غیرمجاز,holiday
0,113201,آزادراه قم - گرمسار (چرمشهر),1400/01/01 00:00:00,1400/01/02 00:00:00,1440.0,1503,1302.0,66.0,15.0,32.0,...,47.0,0.0,1503.0,NaN,NaN,NaN,NaN,NaN,NaN,True
1,113201,آزادراه قم - گرمسار (چرمشهر),1400/01/02 00:00:00,1400/01/03 00:00:00,1380.0,1589,1349.0,74.0,26.0,50.0,...,51.0,0.0,1614.0,NaN,NaN,NaN,NaN,NaN,NaN,True
2,113201,آزادراه قم - گرمسار (چرمشهر),1400/01/03 00:00:00,1400/01/04 00:00:00,1440.0,1767,1408.0,138.0,34.0,59.0,...,57.0,0.0,1767.0,NaN,NaN,NaN,NaN,NaN,NaN,True
3,113201,آزادراه قم - گرمسار (چرمشهر),1400/01/04 00:00:00,1400/01/05 00:00:00,1430.0,2221,1709.0,196.0,51.0,89.0,...,79.0,0.0,2233.0,NaN,NaN,NaN,NaN,NaN,NaN,True
11,113201,آزادراه قم - گرمسار (چرمشهر),1400/01/12 00:00:00,1400/01/13 00:00:00,1440.0,3298,2501.0,219.0,107.0,141.0,...,155.0,0.0,3298.0,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,114552,آزادراه کرج - آزادگان (عوارضي),1395/01/02 00:00:00,1395/01/03 00:00:00,NaN,5686,5414.0,172.0,2.0,21.0,...,NaN,NaN,NaN,1365.0,NaN,NaN,1310.0,0.0,0.0,True
2,114552,آزادراه کرج - آزادگان (عوارضي),1395/01/03 00:00:00,1395/01/04 00:00:00,NaN,6163,5796.0,223.0,3.0,21.0,...,NaN,NaN,NaN,1440.0,NaN,NaN,832.0,1.0,0.0,True
3,114552,آزادراه کرج - آزادگان (عوارضي),1395/01/04 00:00:00,1395/01/05 00:00:00,NaN,5982,5186.0,659.0,5.0,29.0,...,NaN,NaN,NaN,1425.0,NaN,NaN,1239.0,0.0,0.0,True
11,114552,آزادراه کرج - آزادگان (عوارضي),1395/01/12 00:00:00,1395/01/13 00:00:00,NaN,8607,7700.0,697.0,14.0,50.0,...,NaN,NaN,NaN,1440.0,NaN,NaN,1062.0,0.0,0.0,True


In [ ]:
# TODO
X_train = []

In [ ]:
clf.fit(X_train)

# get the prediction label and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# it is possible to get the prediction confidence as well
y_test_pred, y_test_pred_confidence = clf.predict(X_test, return_confidence=True)  # outlier labels (0 or 1) and confidence in the range of [0,1]

In [ ]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

In [ ]:
visualize(clf_name, X_train, y_train, X_test, y_test, y_train_pred,
    y_test_pred, show_figure=True, save_figure=False)

In [ ]:
# save the model
dump(clf, 'clf.joblib')
# load the model
clf = load('clf.joblib')